In [2]:
import pandas as pd
import plotly.express as px
from src import config
from tqdm import tqdm
import pandas as pd

gpt = pd.read_parquet("output/predictions_gpt_4_1.parquet")
gptMini = pd.read_parquet("output/predictions_gpt_4_1_mini.parquet")

dc = pd.read_parquet("output/predictions_deepseek_chat.parquet")
dcR = pd.read_parquet("output/predictions_deepseek_reasoner.parquet")



In [8]:

import pandas as pd
from sklearn.metrics import accuracy_score, f1_score

y_true = dcR['ground_truth']
y_pred = dcR['prediction']

# --- Calculate Metrics ---

# 1. Accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# For multiclass classification, you should specify the 'average' parameter.
# 'macro' calculates the metric for each label, and finds their unweighted mean.
f1_macro = f1_score(y_true, y_pred, average='macro')
print(f"F1 Score (Macro): {f1_macro:.4f}")

# 3. Weighted F1 Score
# This calculates the metric for each label, and finds their average,
# weighted by the number of true instances for each label (the support).
f1_weighted = f1_score(y_true, y_pred, average='weighted')
print(f"F1 Score (Weighted): {f1_weighted:.4f}")

Accuracy: 0.5168
F1 Score (Macro): 0.4254
F1 Score (Weighted): 0.5606


In [31]:
dc.groupby("vote_correct")["discipline"].mean()

vote_correct
False    0.972166
True     0.986561
Name: discipline, dtype: float64

In [ ]:
from scipy import stats
discipline_true = dc[dc["vote_correct"] == True]["discipline"]
discipline_false = dc[dc["vote_correct"] == False]["discipline"]
t_statistic, p_value = stats.ttest_ind(discipline_true, discipline_false, equal_var=False)
# --- 4. Ergebnisse ausgeben --- 
print(f"t-Statistik: {t_statistic:.4f}")
print(f"p-Wert: {p_value}")

t-Statistik: 6.3589
p-Wert: 2.329985795012167e-10


In [15]:
import math

x_mean = discipline_true.mean()
x_std = discipline_true.std()
y_mean = discipline_false.mean()
y_std = discipline_false.std()

# Get the number of samples in each group
n_x = len(discipline_true)
n_y = len(discipline_false)

# Calculate the t-statistic manually using the formula for Welch's t-test
numerator = x_mean - y_mean
denominator = math.sqrt((x_std**2 / n_x) + (y_std**2 / n_y))
t_statistic_manual = numerator / denominator

print(f"Manual T-statistic: {t_statistic_manual}")

Manual T-statistic: 6.358891035688536


In [ ]:
x_std

np.float64(0.05654076642623574)

: 

In [30]:
# This is the formula for the degrees of freedom when variances are not assumed to be equal
s1_squared = x_std**2
s2_squared = y_std**2

numerator_df = ((s1_squared / n_x) + (s2_squared / n_y))**2
denominator_df = ( (s1_squared / n_x)**2 / (n_x - 1) ) + ( (s2_squared / n_y)**2 / (n_y - 1) )

welch_df = numerator_df / denominator_df
print(f"Welch's Degrees of Freedom: {welch_df}")

# --- Step 2: Calculate the one-tailed p-value ---
# We use the survival function (sf), which is 1 - cdf.
# This gives the area in the right tail of the distribution, which is
# exactly what we want for the hypothesis that x_mean > y_mean.
p_value_one_tailed = stats.t.sf(abs(6.36), df=welch_df)

p_value_one_tailed

Welch's Degrees of Freedom: 3108.4294460421056


np.float64(1.1567203789548698e-10)

In [29]:
welch_df

np.float64(3108.4294460421056)

In [22]:
x_mean

np.float64(0.9865608406243725)

In [26]:
len(gpt)-2

3460